# Hazardous Material Sign Detection

### Install packages

In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
from ultralytics import YOLO
import random
import os
import matplotlib.pyplot as plt

Get path

In [ ]:
path = !pwd
path = path[0]

### Dataset

The hazmat labels of the RoboCupRescue League are:
* explosives
* blasting agents
* flammable gas
* non flammable gas
* oxygen
* fuel-oil
* dangerous-when-wet
* flammable-solid
* spontaneously-combistible
* oxidizer
* organic-peroxide
* inhalation-hazard
* poison
* radioactive
* corrosive

The dataset has been downloaded as it is, no further changes have been made, from Roboflow Universe (https://universe.roboflow.com/kauil/hazmat-h6fgb). It is called ```HazMat Computer Vision Project``` by ```Kauil``` licensed by ```Creative Commons BY 4.0```, the format for ```YOLOv11``` has been downloaded

In [ ]:
!unzip -n ./dataset/HazMat.v16i.yolov11 -d dataset

### Structure of the directory
```
.
└── hazmat_detection/
    ├── checkpoints/
    │   └── ...
    ├── dataset/
    │   ├── data.yaml
    │   │
    │   ├── train/
    │   │   ├── images/
    │   │   │
    │   │   └── labels/
    │   │  
    │   ├── val/
    │   │
    │   ├── test/
    │   │ 
    │   └── ...
    ├── utils/
    │   └── ...
    ├── model.ipynb
    └── requirements.txt
```

## Data distribution

Check data distribution in the training, validation and test

In [ ]:
!python ./utils/data_exploration.py

In [ ]:
image = plt.imread("./utils/frequency_class.png", format="png")
plt.imshow(image)
plt.axis("off")
plt.show()

## Train model

In [ ]:
TRAIN = 1 # 1 if training, 0 if inference
model_path = "./checkpoints/yolo11n.pt" if TRAIN else "./runs/detect/train/weights/best.pt" # path to trained model
epochs = 30
batch_size = 64
IMAGE_SIZE = 640 # size for YOLO model

Check if CUDA available, otherwise try with MPS and then CPU

In [ ]:
if torch.cuda.is_available():
    print("Cuda")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Cuda not available, use mps")
    device = torch.device("mps")
else:
    print("Use CPU")
    device = torch.device("cpu")

Load model

In [ ]:
# load pretrained model
model = YOLO(model_path)

Train

In [ ]:
results = model.train(data=f"{path}/dataset/data.yaml", epochs=epochs, batch=batch_size, device=device,
        imgsz=IMAGE_SIZE,
        augment=True,
        flipud=0.5,    # Probability for vertical flip
        fliplr=0.5,    # Probability for horizontal flip
        hsv_h=0.01,    # Adjust hue to all images
        hsv_s=0.01,     # Adjust saturation
        hsv_v=0.01,     # Adjust value
        degrees=180, # rotate image
        translate=0.7, # translate image
        scale=0.7, # scale image
        shear=20, # mimic image seen by different angles
        erasing=0.2, # erase some part of the image
        perspective=0.0005, # see image from different angles
        crop_fraction=0.4,
    )

Validation

In [ ]:
results = model.val(
        data=f"{path}/dataset/data.yaml",
        epochs=epochs,
        batch=batch_size,
        imgsz=IMAGE_SIZE,
        device=device
    )

## Test model

In [ ]:
pathTest = f"{path}/dataset/test/images"
filenames = os.listdir(pathTest)
random.shuffle(filenames)

for filename in filenames:
    path = os.path.join(pathTest, filename)
    results = model.predict(path)
    for result in results:
        result.show()

## Save model

While training, YOLO Ultralytics creates a folder called ```runs``` where you can find the best and last model trained. You can download them and use them as starting point for finetuning, further training and so on.

## **Exporting**
- Run `python3 exporter.py` in terminal to export the model as `.engine` file in order to use it in the `detector.py` node.